# Antes de empezar:
- Lee el archivo README.md
- Comenta tanto como puedas y utiliza los recursos (archivo README.md)
- ¡Feliz aprendizaje!

In [56]:
# import numpy and pandas
import pandas as pd
import numpy as np
from scipy.stats import trim_mean, mode, skew, gaussian_kde, pearsonr, spearmanr, beta
from statsmodels.stats.weightstats import ztest as ztest

from scipy.stats import ttest_ind, norm, t,ttest_1samp 
from scipy.stats import f_oneway
from scipy.stats import sem

# Reto 1 - Explorando los Datos

En este reto, examinaremos todos los salarios de los empleados de la Ciudad de Chicago. Comenzaremos cargando el conjunto de datos y examinando su contenido.

In [57]:
# Run this code:
salaries = pd.read_csv('..\data\Current_Employee_Names__Salaries__and_Position_Titles.csv')
#\\wsl.localhost\Ubuntu\home\ubuntu\my-labs\21-lab-hypothesis-testing-es\data\Current_Employee_Names__Salaries__and_Position_Titles.csv
#\\wsl.localhost\Ubuntu\home\ubuntu\my-labs\21-lab-hypothesis-testing-es\data\Current_Employee_Names__Salaries__and_Position_Titles.csv

<>:2: SyntaxWarning: invalid escape sequence '\d'
<>:2: SyntaxWarning: invalid escape sequence '\d'
C:\Users\Aleja\AppData\Local\Temp\ipykernel_16676\3272400965.py:2: SyntaxWarning: invalid escape sequence '\d'
  salaries = pd.read_csv('..\data\Current_Employee_Names__Salaries__and_Position_Titles.csv')


Examina el conjunto de datos `salaries` utilizando la función `head` que se muestra a continuación.

In [58]:
# Your code here
print(salaries.info())
print(salaries.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33183 entries, 0 to 33182
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               33183 non-null  object 
 1   Job Titles         33183 non-null  object 
 2   Department         33183 non-null  object 
 3   Full or Part-Time  33183 non-null  object 
 4   Salary or Hourly   33183 non-null  object 
 5   Typical Hours      8022 non-null   float64
 6   Annual Salary      25161 non-null  float64
 7   Hourly Rate        8022 non-null   float64
dtypes: float64(3), object(5)
memory usage: 2.0+ MB
None
       Typical Hours  Annual Salary  Hourly Rate
count    8022.000000   25161.000000  8022.000000
mean       34.507604   86786.999790    32.788558
std         9.252077   21041.354602    12.112573
min        10.000000    7200.000000     2.650000
25%        20.000000   76266.000000    21.200000
50%        40.000000   90024.000000    35.600000
75%        4

# Reto 2 - Pruebas de Hipótesis

En esta sección del laboratorio, vamos a probar si el salario por hora de todos los trabajadores por hora es significativamente diferente de $30/hora. Importa la función correcta para la prueba de una muestra desde scipy y realiza la prueba de hipótesis para un intervalo de confianza de dos lados del 95%.

In [67]:
# Your code here
# Definir el nivel de significancia (alpha)
alpha = 0.05

# Extraer los salarios por hora del DataFrame (suponiendo que 'Hourly Rate' contiene estos datos)
hourly_rates = salaries['Hourly Rate'].dropna()  # Eliminar NaN si los hubiera

# Realizar la prueba de hipótesis
t_statistic, p_value = ttest_1samp(a=hourly_rates, popmean= 30)

# Imprimir los resultados
print("Prueba de hipótesis de una muestra:")
print(f"Estadístico t: {t_statistic}")
print(f"Valor p: {p_value}")

# Comparar el valor p con el nivel de significancia
if p_value < alpha:
    print("Podemos rechazar la hipótesis nula (H0). El salario por hora es significativamente diferente de $30/hora.")
else:
    print("No podemos rechazar la hipótesis nula (H0). No hay evidencia suficiente para concluir que el salario por hora difiere significativamente de $30/hora.")

Prueba de hipótesis de una muestra:
Estadístico t: 20.6198057854942
Valor p: 4.3230240486229894e-92
Podemos rechazar la hipótesis nula (H0). El salario por hora es significativamente diferente de $30/hora.


# Reto 3 - Construyendo Intervalos de Confianza

Aunque probar nuestra hipótesis es una excelente manera de obtener evidencia empírica para aceptar o rechazar la hipótesis, otra forma de recopilar evidencia es creando un intervalo de confianza. Un intervalo de confianza nos brinda información sobre la media verdadera de la población. Por lo tanto, para un intervalo de confianza del 95%, estamos seguros en un 95% de que la media de la población se encuentra dentro del intervalo de confianza.

Para leer más sobre los intervalos de confianza, haz clic [aquí](https://es.wikipedia.org/wiki/Intervalo_de_confianza).

En la celda a continuación, construiremos un intervalo de confianza del 95% para el salario por hora medio de todos los trabajadores por hora.

El intervalo de confianza se calcula en SciPy utilizando la función `t.interval`. Puedes leer más sobre esta función [aquí](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

Para calcular el intervalo de confianza del salario por hora, usa 0.95 para el nivel de confianza, el número de filas - 1 para los grados de libertad, la media de la muestra para el parámetro de ubicación y el error estándar para la escala. El error estándar se puede calcular utilizando [esta](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) función en SciPy.

In [60]:
# Your code here
df_temporal= salaries['Hourly Rate'].copy()
df_temporal= df_temporal.dropna()

In [61]:
n = len(df_temporal)
df = n - 1

# Calcular el margen de error
t_critico = t.ppf(0.975, df)  # Prueba de dos colas, 95% de confianza
margen_error = t_critico * (np.std(df_temporal) / np.sqrt(n))

# Calcular el intervalo de confianza
intervalo_confianza = (np.mean(df_temporal) - margen_error, np.mean(df_temporal) + margen_error)

print("Intervalo de confianza del 95%:", intervalo_confianza)

Intervalo de confianza del 95%: (32.52347486867093, 33.053640563889545)


# Reto 4 - Pruebas de Hipótesis de Proporciones

Otro tipo de prueba de una muestra es una prueba de hipótesis de proporciones. En esta prueba, examinamos si la proporción de un grupo en nuestra muestra es significativamente diferente de una fracción.

Puedes leer más sobre las pruebas de proporción de una muestra [aquí](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

En la celda a continuación, utiliza la función `proportions_ztest` de `statsmodels` para realizar una prueba de hipótesis que determinará si el número de trabajadores por hora en la Ciudad de Chicago es significativamente diferente del 25% en el nivel de confianza del 95%.

In [66]:
# Your code here
# Datos de ejemplo
from statsmodels.stats.proportion import proportions_ztest

# Datos de ejemplo (reemplaza con tus valores reales)
count = len(salaries[salaries['Salary or Hourly'] == 'Hourly'])  # Número de trabajadores por hora
nobs = len(salaries)  # Número total de empleados

# Proporción hipotética a probar (25%)
proporcion_nula = 0.25

# Realizar la prueba de proporciones
z_statistic, p_value = proportions_ztest(count,nobs,proporcion_nula)

# Nivel de significancia (para un intervalo de confianza del 95%)
alpha = 0.05

# Imprimir resultados
print(f'Estadístico z: {z_statistic:.4f}')
print(f'Valor p: {p_value:.4f}')

# Evaluar la hipótesis nula
if p_value < alpha:
    print("Rechazamos la hipótesis nula. Hay evidencia suficiente para concluir que la proporción de trabajadores por hora en Chicago es significativamente diferente del 25%.")
else:
    print("No rechazamos la hipótesis nula. No hay evidencia suficiente para concluir que la proporción de trabajadores por hora en Chicago es significativamente diferente del 25%.")





Estadístico z: -3.5100
Valor p: 0.0004
Rechazamos la hipótesis nula. Hay evidencia suficiente para concluir que la proporción de trabajadores por hora en Chicago es significativamente diferente del 25%.
